In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Define the path to the directory
directory_path = r'C:\Users\shaif\Downloads\Compressed\Invasive_Life_Stage'

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for folder in os.listdir(directory_path):
    #print(class_folder)
    clsp = os.path.join(directory_path,folder)  
    for class_folder in os.listdir(clsp):
        #print(class_folder)
        class_path = os.path.join(clsp, class_folder)    
        #print(class_path)
        for image_file in os.listdir(class_path):
            if image_file.endswith('.png'):
                image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
                img = Image.open(image_path).convert('RGB')
                img = img.resize((32, 32))
                img_array = np.array(img)
            
            # Append image and label to lists
                X_train.append(img_array)
                Y_train.append(class_label)
                #print(class_label)
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')


C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


X_train shape: (49227, 32, 32, 3)
Y_train shape: (49227,)


In [2]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Y_train_encoded = to_categorical(Y_train)
# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train_encoded, test_size=0.2, random_state=42)

In [ ]:
from tensorflow import keras
from keras.models import load_model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D

model_path = r"C:\Users\shaif\Downloads\mob_invasive_ssl.h5"
model = load_model(model_path)
#model.layers[0].trainable = False
x = model.layers[-8].output  # Access the last 4th layer from the end
output = Dense(5, activation='softmax')(x)
new_model = keras.models.Model(inputs=model.input, outputs=output)
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
new_model.fit(np.array(X_train).astype('float32'),y_train, batch_size=32, epochs=100, validation_split = .2)

Epoch 1/100
985/985 [==============================] - 42s 38ms/step - loss: 27.3471 - accuracy: 0.4359 - val_loss: 24.2960 - val_accuracy: 0.4389
Epoch 2/100
985/985 [==============================] - 34s 35ms/step - loss: 12.8967 - accuracy: 0.4493 - val_loss: 14.4503 - val_accuracy: 0.3405
Epoch 3/100
985/985 [==============================] - 38s 38ms/step - loss: 10.3718 - accuracy: 0.4635 - val_loss: 1.8183 - val_accuracy: 0.4753
Epoch 4/100
985/985 [==============================] - 32s 32ms/step - loss: 9.4561 - accuracy: 0.4648 - val_loss: 1.6582 - val_accuracy: 0.4837
Epoch 5/100
985/985 [==============================] - 40s 41ms/step - loss: 8.5413 - accuracy: 0.4641 - val_loss: 1.2166 - val_accuracy: 0.4142
Epoch 6/100
985/985 [==============================] - 38s 39ms/step - loss: 7.6878 - accuracy: 0.4702 - val_loss: 1.6192 - val_accuracy: 0.4749
Epoch 7/100
985/985 [==============================] - 45s 46ms/step - loss: 8.7282 - accuracy: 0.4786 - val_loss: 1.1831 - v

985/985 [==============================] - 40s 41ms/step - loss: 1.7956 - accuracy: 0.4213 - val_loss: 1.2915 - val_accuracy: 0.3814
Epoch 57/100
985/985 [==============================] - 47s 48ms/step - loss: 1.3564 - accuracy: 0.4772 - val_loss: 1.3482 - val_accuracy: 0.3871
Epoch 58/100
404/985 [===========>..................] - ETA: 20s - loss: 1.2548 - accuracy: 0.5053

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
balanced_accuracy = balanced_accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Balanced Accuracy:", balanced_accuracy)
print("Test Accuracy:", accuracy)

In [33]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load MobileNetV2 model without top layer
base_model = MobileNetV2(include_top=False, weights=None, input_shape=(32,32,3), pooling="avg")
x =  Dense(500)(base_model.output)
# Add custom top layers
output = Dense(5, activation='softmax')(x)  # Update num_classes with your number of classes
# Create the new model
ne_model = Model(inputs=base_model.input, outputs=output)
# Compile the model
ne_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
ne_model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

Epoch 1/100
985/985 [==============================] - 40s 36ms/step - loss: 0.8048 - accuracy: 0.7427 - val_loss: 3.4905 - val_accuracy: 0.5420
Epoch 2/100
985/985 [==============================] - 38s 38ms/step - loss: 0.9070 - accuracy: 0.6952 - val_loss: 1.4212 - val_accuracy: 0.5050
Epoch 3/100
985/985 [==============================] - 46s 47ms/step - loss: 0.6569 - accuracy: 0.7597 - val_loss: 0.6924 - val_accuracy: 0.7569
Epoch 4/100
985/985 [==============================] - 38s 39ms/step - loss: 0.6276 - accuracy: 0.7750 - val_loss: 1.9463 - val_accuracy: 0.5611
Epoch 5/100
985/985 [==============================] - 43s 43ms/step - loss: 0.5192 - accuracy: 0.8143 - val_loss: 1.4517 - val_accuracy: 0.7217
Epoch 6/100
985/985 [==============================] - 42s 42ms/step - loss: 0.4746 - accuracy: 0.8305 - val_loss: 4.0191 - val_accuracy: 0.4836
Epoch 7/100
985/985 [==============================] - 38s 39ms/step - loss: 0.6172 - accuracy: 0.7769 - val_loss: 0.8028 - val_ac

Epoch 56/100
985/985 [==============================] - 40s 40ms/step - loss: 0.1103 - accuracy: 0.9617 - val_loss: 0.7177 - val_accuracy: 0.8455
Epoch 57/100
985/985 [==============================] - 36s 36ms/step - loss: 0.1036 - accuracy: 0.9639 - val_loss: 0.5247 - val_accuracy: 0.8563
Epoch 58/100
985/985 [==============================] - 42s 43ms/step - loss: 0.1514 - accuracy: 0.9490 - val_loss: 0.5352 - val_accuracy: 0.8215
Epoch 59/100
985/985 [==============================] - 41s 41ms/step - loss: 0.1058 - accuracy: 0.9642 - val_loss: 0.3347 - val_accuracy: 0.8960
Epoch 60/100
985/985 [==============================] - 38s 39ms/step - loss: 0.1278 - accuracy: 0.9545 - val_loss: 0.3406 - val_accuracy: 0.8889
Epoch 61/100
985/985 [==============================] - 38s 38ms/step - loss: 0.1085 - accuracy: 0.9616 - val_loss: 0.3665 - val_accuracy: 0.8788
Epoch 62/100
985/985 [==============================] - 37s 38ms/step - loss: 0.0925 - accuracy: 0.9677 - val_loss: 0.3664 -

In [28]:
from sklearn.metrics import accuracy_score
predictions = ne_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Accuracy:", accuracy)

Test Accuracy: 0.6181190331098924


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load MobileNetV2 model without top layer
base_model = MobileNetV2(include_top=False, weights=None, input_shape=(32,32,3), pooling="avg")
x =  Dense(500)(base_model.output)
# Add custom top layers
output = Dense(5, activation='softmax')(x)  # Update num_classes with your number of classes

# Create the new model
new_model = Model(inputs=base_model.input, outputs=output)

# Compile the model
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
new_model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)


In [ ]:
from sklearn.metrics import accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Accuracy:", accuracy)

In [26]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model

# Load the original model
model_path = r"C:\Users\shaif\Downloads\mob_invasive_ssl.h5"
model = load_model(model_path)

# Add a global average pooling layer after the desired layer
x = model.layers[-8].output  # Access the layer you want to connect to
x =  Dense(500)(x)

# Define the new output layer connected to the global average pooling layer
output = Dense(5, activation='softmax', name='output_layer')(x)

# Create the new model with the updated head
new_model = Model(inputs=model.input, outputs=output)

# Now you should be able to view the summary without errors
new_model.summary()


Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 1280)             2257984   
 ional)                                                          
                                                                 
 dense_1 (Dense)             (None, 2048)              2623488   
                                                                 
 dense_12 (Dense)            (None, 500)               1024500   
                                                                 
 output_layer (Dense)        (None, 5)                 2505      
                                                                 
Total params: 5,908,477
Trainable params: 5,874,365
Non-trainable params: 34,112
___________________________________________

In [8]:
from tensorflow.keras.applications import MobileNetV2
base_model = MobileNetV2(include_top=False, weights="imagenet", input_shape=(32,32,3), pooling="avg")
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 16, 16, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 16, 16, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 16, 16, 32)   0           ['bn_Conv1[0][0]']               
                                                                               

 block_3_depthwise (DepthwiseCo  (None, 4, 4, 144)   1296        ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 4, 4, 144)   576         ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_depthwise_relu (ReLU)  (None, 4, 4, 144)    0           ['block_3_depthwise_BN[0][0]']   
                                                                                                  
 block_3_project (Conv2D)       (None, 4, 4, 32)     4608        ['block_3_depthwise_relu[0][0]'] 
                                                                                                  
 block_3_p

 block_7_expand_BN (BatchNormal  (None, 2, 2, 384)   1536        ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 2, 2, 384)    0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 2, 2, 384)   3456        ['block_7_expand_relu[0][0]']    
 nv2D)                                                                                            
                                                                                                  
 block_7_depthwise_BN (BatchNor  (None, 2, 2, 384)   1536        ['block_7_depthwise[0][0]']      
 malization)                                                                                      
          

                                                                                                  
 block_10_project_BN (BatchNorm  (None, 2, 2, 96)    384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 2, 2, 576)    55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_expand_BN (BatchNorma  (None, 2, 2, 576)   2304        ['block_11_expand[0][0]']        
 lization)                                                                                        
                                                                                                  
 block_11_expand_relu (ReLU)    (None, 2, 2, 576)    0           ['block_11_expand_BN[0][0]']     
          

 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 1, 1, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 1, 1, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_project_BN (BatchNorm  (None, 1, 1, 160)   640         ['block_14_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_14_add (Add)             (None, 1, 1, 160)    0           ['block_13_project_BN[0][0]',    
          